In [1]:
import sqlite3
import pandas
from sqlite3 import Error
import json

/tmp/ipykernel_104524/125094152.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


In [3]:
# https://www.kaggle.com/datasets/shiivvvaam/bitcoin-historical-data
df = pandas.read_csv('Bitcoin History.csv')

In [3]:
df

,Date,Price,Open,High,Low,Vol.,Change %
0,"Feb 09, 2024","47,545.4","45,293.3","47,710.2","45,254.2",86.85K,4.97%
1,"Feb 08, 2024","45,293.3","44,346.2","45,579.2","44,336.4",66.38K,2.15%
2,"Feb 07, 2024","44,339.8","43,088.4","44,367.9","42,783.5",48.57K,2.91%
3,"Feb 06, 2024","43,087.7","42,697.6","43,375.5","42,566.8",33.32K,0.91%
4,"Feb 05, 2024","42,697.2","42,581.4","43,532.2","42,272.5",39.26K,0.27%
...,...,...,...,...,...,...,...
4950,"Jul 22, 2010",0.1,0.1,0.1,0.1,2.16K,0.00%
4951,"Jul 21, 2010",0.1,0.1,0.1,0.1,0.58K,0.00%
4952,"Jul 20, 2010",0.1,0.1,0.1,0.1,0.26K,0.00%
4953,"Jul 19, 2010",0.1,0.1,0.1,0.1,0.57K,0.00%


In [4]:
df = df.drop('Vol.', axis=1)

In [5]:
df['Date'] = pandas.to_datetime(df['Date'])
df['Price'] = pandas.to_numeric(df['Price'].str.replace(',',''))
df['Open'] = pandas.to_numeric(df['Open'].str.replace(',',''))
df['High'] = pandas.to_numeric(df['High'].str.replace(',',''))
df['Low'] = pandas.to_numeric(df['Low'].str.replace(',',''))

df['Change %'] = pandas.to_numeric(df['Change %'].str.replace('%',''))/100

In [4]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
        return conn
    except Error as e:
        print(e)
    # finally:
    #     if conn:
    #         conn.close()


if __name__ == '__main__':
    conn = create_connection(r"./sql.db")
    if conn:
        df.to_sql('Bitcoin_History', conn)
        conn.close()
        

2.6.0


ValueError: Table 'Bitcoin_History' already exists.

In [7]:
df

,Date,Price,Open,High,Low,Change %
0,2024-02-09,47545.4,45293.3,47710.2,45254.2,0.0497
1,2024-02-08,45293.3,44346.2,45579.2,44336.4,0.0215
2,2024-02-07,44339.8,43088.4,44367.9,42783.5,0.0291
3,2024-02-06,43087.7,42697.6,43375.5,42566.8,0.0091
4,2024-02-05,42697.2,42581.4,43532.2,42272.5,0.0027
...,...,...,...,...,...,...
4950,2010-07-22,0.1,0.1,0.1,0.1,0.0000
4951,2010-07-21,0.1,0.1,0.1,0.1,0.0000
4952,2010-07-20,0.1,0.1,0.1,0.1,0.0000
4953,2010-07-19,0.1,0.1,0.1,0.1,0.0000


In [6]:
conn = sqlite3.connect('sql.db')
cur = conn.cursor()
cur.execute("PRAGMA table_info(Bitcoin_History)")

rows = cur.fetchall()

for row in rows:
    print(row)
# print(conn.execute('SELECT * FROM Bitcoin_History'))

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'Date', 'TIMESTAMP', 0, None, 0)
(2, 'Price', 'REAL', 0, None, 0)
(3, 'Open', 'REAL', 0, None, 0)
(4, 'High', 'REAL', 0, None, 0)
(5, 'Low', 'REAL', 0, None, 0)
(6, 'Change', 'REAL', 0, None, 0)


In [11]:
def get_table_metadata(table_name, db_name, metadata_name):
    conn = sqlite3.connect(db_name)
    cur = conn.cursor()
    table_name = table_name
    cur.execute(f"PRAGMA table_info({table_name})")

    rows = cur.fetchall()

    schema = ""
    with open(metadata_name) as f:
        comment_dict = json.load(f)

        for row in rows:
            schema += f'\t{row[1]} {row[2]}, -- {comment_dict[row[1]]}\n'

    return schema

print(get_table_metadata("Bitcoin_History", "sql.db", "sqldb.json"))

	index INTEGER, -- Unique ID for each product
	Date TIMESTAMP, -- Date of the recorded data
	Price REAL, -- Closing price of Bitcoin on the given date
	Open REAL, -- Opening price of Bitcoin on the given date
	High REAL, -- Highest price of Bitcoin on the given date
	Low REAL, -- Lowest price of Bitcoin on the given date
	Change REAL, -- Percentage change in Bitcoin's price from the previous day. stored as decimal. not percent.



In [ ]:


#ensure no edit commands are passed in





conn.close()